In [12]:
import torch
import numpy as np
from torchvision.datasets import MNIST
from torchvision.transforms import transforms
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
os.chdir(os.path.dirname(os.path.abspath("")))
print("current working directory:", os.getcwd())

from utils.dataloaders.full_dataloaders import DataLoaderMNIST, DataLoaderFashionMNIST, DataLoaderCIFAR10, DataLoaderCIFAR100
from utils.visualization import (
    visualize_mapping_error,
    visualize_latent_space_pca,
    plot_latent_space,
    highlight_cluster,
)
from utils.model import load_model, get_transformations

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

current working directory: /mnt/c/Users/Hillary Hauger/Documents/Studium/SoSe2024/Case Study Non Linear Optimization/Code


In [14]:
def get_stitched_output(model1, model2, mapping, images):
    latent_space1 = model1.get_latent_space(images).to(dtype=torch.float32)
    latent_space_stitched = mapping.predict(latent_space1.detach().cpu())
    #Convert to tensor if necessary and to the right dtype
    if isinstance(latent_space_stitched, np.ndarray):
        latent_space_stitched = torch.tensor(latent_space_stitched, dtype=torch.float32)
    elif isinstance(latent_space_stitched, torch.Tensor):
        latent_space_stitched = latent_space_stitched.to(dtype=torch.float32)
    
    outputs = model2.decode(latent_space_stitched.to(DEVICE))
    return outputs

def simple_sampler(indices, model, transformations, device, seed=10):
    """
    Input:
    - model: Model
    - indices: Indices of the dataset
    - transformations: Transformations to be applied to the images
    Output:
    - z: Latent vectors of the model
    - labels: Labels of the dataset

    This function samples the latent space of the model and returns the latent vectors
    """
    data_loader = DataLoaderMNIST(128, transformations, seed=seed, indices=indices)
    train_loader = data_loader.get_train_loader()

    # get all images from train_loader and convert them to latent space
    all_images = []
    all_latent_space=[]
    for images, _ in train_loader:
        images = images.to(device)
        latent_space = model.get_latent_space(images)
        all_latent_space.append(latent_space)
        all_images.append(images)
    images = torch.cat(all_images, dim=0)
    z = torch.cat(all_latent_space, dim=0)
    z = z.detach().cpu().numpy()
    labels = data_loader.train_loader.dataset.dataset.targets[indices]
    return z,images,labels

In [3]:
name_model = "VerySmall-AE"
name_dataset = "MNIST"
size_of_the_latent = 10
seed1,seed2=1,2
num_samples = 200
model_path1 = f"models/checkpoints/VerySmallAE/{name_dataset}/{name_dataset}_{name_model}_{size_of_the_latent}_{seed1}.pth"
model_path2 = f"models/checkpoints/VerySmallAE/{name_dataset}/{name_dataset}_{name_model}_{size_of_the_latent}_{seed2}.pth"
model1 = load_model(model_name=name_model, name_dataset=name_dataset, latent_size=int(size_of_the_latent), seed=int(seed1), model_path = model_path1)
model2 = load_model(model_name=name_model, name_dataset=name_dataset, latent_size=int(size_of_the_latent), seed=int(seed2), model_path = model_path2)

In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
print("Using device (CPU/GPU):", device)
transformations = get_transformations(name_model)
indices = np.random.randint(0, 10000, num_samples)
latent_space1, images,labels = simple_sampler(indices, model1, transformations, device, seed=10)
latent_space2, images,labels = simple_sampler(indices, model2, transformations, device, seed=10)

Using device (CPU/GPU): cpu


In [5]:
print(latent_space1.shape)
print(latent_space2.shape)

(200, 10)
(200, 10)


In [6]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split

X= latent_space1
y =latent_space2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

kr = KernelRidge(kernel='rbf', gamma=0.1, alpha=1.0)
kr.fit(X_train,y_train)

print("Dual coefficients (alpha):")
print(kr.dual_coef_.shape)

# Make predictions
predictions = kr.predict(X_train)
print("Predictions error on train set:")
print(np.linalg.norm(predictions-y_train))
predictions = kr.predict(X_test)
print("Predictions error on test set")
print(np.linalg.norm(predictions-y_test))

Dual coefficients (alpha):
(160, 10)
Predictions error on train set:
44.140335
Predictions error on test set
46.823223


In [7]:
from sklearn.linear_model import LinearRegression
# Create and fit the Linear Regression model
lr = LinearRegression()
lr.fit(X_train, y_train)

print("Coefficent:")
print(lr.coef_.shape)

# Make predictions
predictions = lr.predict(X_train)
print("Predictions error on train set:")
print(np.linalg.norm(predictions-y_train))
predictions = lr.predict(X_test)
print("Predictions error on test set")
print(np.linalg.norm(predictions-y_test))

Coefficent:
(10, 10)
Predictions error on train set:
23.043707
Predictions error on test set
13.790824


In [8]:
from sklearn.preprocessing import PolynomialFeatures

# Generate example data

# Polynomial regression setup (degree 2 for example)
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly_train = poly.fit_transform(X_train)
X_poly_test = poly.transform(X_test)

# Create and fit the Polynomial Regression model
lr_poly = LinearRegression()
lr_poly.fit(X_poly_train, y_train)

print("Polynomial Regression coefficients:")
print(lr_poly.coef_.shape)

# Make predictions
predictions = lr_poly.predict(X_poly_train)
print("Predictions error on train set:")
print(np.linalg.norm(predictions-y_train))
predictions = lr_poly.predict(X_poly_test)
print("Predictions error on test set")
print(np.linalg.norm(predictions-y_test))

Polynomial Regression coefficients:
(10, 65)
Predictions error on train set:
13.207867
Predictions error on test set
15.442214


In [11]:
from sklearn.preprocessing import SplineTransformer
from sklearn.pipeline import make_pipeline

# Create a spline transformer
spline_transformer = SplineTransformer(degree=3, n_knots=5, include_bias=False)

# Create a pipeline with the spline transformer and linear regression
spline_regression = make_pipeline(spline_transformer, LinearRegression())
spline_regression.fit(X_train, y_train)

# Print the shape of the coefficients for Spline Regression
print("\nSpline Regression coefficients shape:")
print(spline_regression.named_steps['linearregression'].coef_.shape)

# Make predictions with Spline Regression
spline_train_predictions = spline_regression.predict(X_train)
spline_test_predictions = spline_regression.predict(X_test)

# Evaluate the Spline Regression model
spline_train_error = np.linalg.norm(spline_train_predictions - y_train)
spline_test_error = np.linalg.norm(spline_test_predictions - y_test)

print("\nSpline Regression Predictions error on train set:")
print(spline_train_error)
print("Spline Regression Predictions error on test set:")
print(spline_test_error)


Spline Regression coefficients shape:
(10, 60)

Spline Regression Predictions error on train set:
16.928808
Spline Regression Predictions error on test set:
16.00588
